# 自然语言生成(NLG)学习笔记

## 目录
1. [什么是NLG](#1-什么是nlg)
2. [双向编码器自回归解码器](#2-双向编码器自回归解码器)
3. [教师强制与学生强制](#3-教师强制与学生强制)
4. [Top-k采样](#4-top-k采样)
5. [练习题](#7-练习题)

## 1. 什么是NLG

### NLG定义
**自然语言生成(Natural Language Generation, NLG)** 是人工智能的一个分支，专注于让计算机能够产生人类可理解的自然语言文本。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## 2. 双向编码器自回归解码器

### 架构概述
**双向编码器自回归解码器(Bidirectional Encoder Autoregressive Decoder)** 是现代NLG模型的主流架构，如BART、T5等。

###  核心组件

#### 双向编码器(Bidirectional Encoder)
- **功能**：理解输入序列的完整上下文
- **特点**：可以同时看到前后文信息
- **实现**：基于Transformer的编码器层

#### 自回归解码器(Autoregressive Decoder)
- **功能**：逐步生成输出序列
- **特点**：只能看到已生成的部分
- **实现**：带掩码的Transformer解码器

###  工作流程
1. **编码阶段**：编码器处理输入序列，生成上下文表示
2. **解码阶段**：解码器基于编码器输出和已生成序列，预测下一个词
3. **交叉注意力**：解码器通过交叉注意力机制关注编码器输出

###  数学表示

编码器：
$$H = \text{Encoder}(X) = \text{BiTransformer}(X)$$

解码器：
$$P(y_t|y_{<t}, H) = \text{Decoder}(y_{<t}, H)$$

完整生成概率：
$$P(Y|X) = \prod_{t=1}^{T} P(y_t|y_{<t}, H)$$

In [3]:
# 简化的编码器-解码器实现
class SimpleEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers):
        super(SimpleEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = nn.Parameter(torch.randn(1000, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        
    def forward(self, x):
        seq_len = x.size(1)
        # 词嵌入 + 位置编码
        embedded = self.embedding(x) + self.pos_encoding[:seq_len]
        # 双向编码
        encoded = self.transformer(embedded)
        return encoded

class SimpleDecoder(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers):
        super(SimpleDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = nn.Parameter(torch.randn(1000, d_model))
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, batch_first=True)
        self.transformer = nn.TransformerDecoder(decoder_layer, num_layers)
        self.output_proj = nn.Linear(d_model, vocab_size)
        
    def forward(self, tgt, memory, tgt_mask=None):
        seq_len = tgt.size(1)
        # 词嵌入 + 位置编码
        embedded = self.embedding(tgt) + self.pos_encoding[:seq_len]
        # 自回归解码
        decoded = self.transformer(embedded, memory, tgt_mask=tgt_mask)
        # 输出投影
        output = self.output_proj(decoded)
        return output

class EncoderDecoderModel(nn.Module):
    def __init__(self, vocab_size, d_model=512, nhead=8, num_layers=6):
        super(EncoderDecoderModel, self).__init__()
        self.encoder = SimpleEncoder(vocab_size, d_model, nhead, num_layers)
        self.decoder = SimpleDecoder(vocab_size, d_model, nhead, num_layers)
        
    def forward(self, src, tgt, tgt_mask=None):
        # 编码输入
        memory = self.encoder(src)
        # 解码输出
        output = self.decoder(tgt, memory, tgt_mask)
        return output
    
    def generate_square_subsequent_mask(self, sz):
        """生成自回归掩码"""
        mask = torch.triu(torch.ones(sz, sz), diagonal=1)
        mask = mask.masked_fill(mask == 1, float('-inf'))
        return mask

# 测试模型
vocab_size = 10000
model = EncoderDecoderModel(vocab_size)

# 模拟数据
batch_size, src_len, tgt_len = 2, 10, 8
src = torch.randint(0, vocab_size, (batch_size, src_len))
tgt = torch.randint(0, vocab_size, (batch_size, tgt_len))
tgt_mask = model.generate_square_subsequent_mask(tgt_len)

output = model(src, tgt, tgt_mask)
print(f"输入形状: {src.shape}")
print(f"目标形状: {tgt.shape}")
print(f"输出形状: {output.shape}")

输入形状: torch.Size([2, 10])
目标形状: torch.Size([2, 8])
输出形状: torch.Size([2, 8, 10000])


## 3. 教师强制与学生强制

###  教师强制(Teacher Forcing)

#### 定义
在训练阶段，使用真实的目标序列作为解码器的输入，而不是使用模型自己的预测结果。

#### 工作原理
- **训练时**：解码器在时间步t使用真实的$y_{t-1}$来预测$y_t$
- **优势**：训练稳定，收敛快速
- **问题**：训练和推理时的输入分布不一致(Exposure Bias)

#### 数学表示
训练时：$P(y_t|y_{<t}^{\text{true}}, X)$

推理时：$P(y_t|y_{<t}^{\text{pred}}, X)$

###  学生强制(Student Forcing)

#### 定义
在训练阶段，使用模型自己的预测结果作为下一步的输入，模拟推理时的情况。

#### 工作原理
- **训练时**：解码器使用自己预测的$\hat{y}_{t-1}$来预测$y_t$
- **优势**：训练和推理一致
- **问题**：训练不稳定，容易发散

### 混合策略

#### 计划采样(Scheduled Sampling)
- **策略**：以概率$p$使用真实标签，以概率$(1-p)$使用模型预测
- **动态调整**：训练过程中逐渐减少$p$

#### 课程学习(Curriculum Learning)
- **从简单到复杂**：先用教师强制，再逐步引入学生强制
- **渐进式训练**：平衡稳定性和一致性

In [5]:
# 教师强制 vs 学生强制实现
class TeacherStudentTrainer:
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
    
    def teacher_forcing_step(self, src, tgt):
        """教师强制训练步骤"""
        self.model.train()
        self.optimizer.zero_grad()
        
        # 使用真实目标序列作为输入(除了最后一个token)
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]
        
        # 生成掩码
        tgt_mask = self.model.generate_square_subsequent_mask(tgt_input.size(1))
        
        # 前向传播
        output = self.model(src, tgt_input, tgt_mask)
        
        # 计算损失
        loss = self.criterion(output.reshape(-1, output.size(-1)), tgt_output.reshape(-1))
        
        # 反向传播
        loss.backward()
        self.optimizer.step()
        
        return loss.item()
    
    def student_forcing_step(self, src, tgt, max_len=50):
        """学生强制训练步骤"""
        self.model.train()
        self.optimizer.zero_grad()
        
        batch_size = src.size(0)
        device = src.device
        
        # 初始化解码器输入(通常是<BOS>标记)
        decoder_input = torch.zeros(batch_size, 1, dtype=torch.long, device=device)
        
        total_loss = 0
        
        # 编码输入
        memory = self.model.encoder(src)
        
        # 逐步生成
        for t in range(min(tgt.size(1) - 1, max_len)):
            # 解码当前步
            tgt_mask = self.model.generate_square_subsequent_mask(decoder_input.size(1))
            output = self.model.decoder(decoder_input, memory, tgt_mask)
            
            # 获取当前步的预测
            current_output = output[:, -1, :]
            
            # 计算损失
            target = tgt[:, t + 1]
            loss = self.criterion(current_output, target)
            total_loss += loss
            
            # 使用模型预测作为下一步输入
            predicted = torch.argmax(current_output, dim=-1, keepdim=True)
            decoder_input = torch.cat([decoder_input, predicted], dim=1)
        
        # 反向传播
        total_loss.backward()
        self.optimizer.step()
        
        return total_loss.item()
    
    def scheduled_sampling_step(self, src, tgt, sampling_prob=0.5):
        """计划采样训练步骤"""
        self.model.train()
        self.optimizer.zero_grad()
        
        batch_size = src.size(0)
        seq_len = tgt.size(1) - 1
        device = src.device
        
        # 编码输入
        memory = self.model.encoder(src)
        
        # 初始化解码器输入
        decoder_input = tgt[:, :1]  # 开始标记
        total_loss = 0
        
        for t in range(seq_len):
            # 解码当前步
            tgt_mask = self.model.generate_square_subsequent_mask(decoder_input.size(1))
            output = self.model.decoder(decoder_input, memory, tgt_mask)
            current_output = output[:, -1, :]
            
            # 计算损失
            target = tgt[:, t + 1]
            loss = self.criterion(current_output, target)
            total_loss += loss
            
            # 计划采样：随机选择使用真实标签还是预测结果
            use_teacher = torch.rand(1).item() < sampling_prob
            
            if use_teacher:
                # 使用真实标签
                next_input = tgt[:, t + 1:t + 2]
            else:
                # 使用模型预测
                next_input = torch.argmax(current_output, dim=-1, keepdim=True)
            
            decoder_input = torch.cat([decoder_input, next_input], dim=1)
        
        # 反向传播
        total_loss.backward()
        self.optimizer.step()
        
        return total_loss.item()

# 示例使用
model = EncoderDecoderModel(vocab_size=1000)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # 忽略padding
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainer = TeacherStudentTrainer(model, criterion, optimizer)

# 模拟数据
src = torch.randint(1, 1000, (4, 10))
tgt = torch.randint(1, 1000, (4, 8))

# 测试不同训练策略
tf_loss = trainer.teacher_forcing_step(src, tgt)
print(f"教师强制损失: {tf_loss:.4f}")

ss_loss = trainer.scheduled_sampling_step(src, tgt, sampling_prob=0.7)
print(f"计划采样损失: {ss_loss:.4f}")

教师强制损失: 7.2767
计划采样损失: 37.1256


## 4. Top-k采样

### 文本生成中的采样策略

#### 贪心解码(Greedy Decoding)
- **策略**：每步选择概率最高的词
- **优点**：确定性，计算简单
- **缺点**：容易陷入重复，缺乏多样性

#### 束搜索(Beam Search)
- **策略**：保持k个最佳候选序列
- **优点**：相对最优解
- **缺点**：仍然缺乏多样性

###  Top-k采样

#### 核心思想
在每个时间步，只从概率最高的k个词中进行采样，过滤掉低概率的词。

#### 算法步骤

1. **计算概率分布**：$P(w|context)$
2. **选择Top-k**：保留概率最高的k个词
3. **重新归一化**：对选中的k个词重新计算概率
4. **随机采样**：从重新归一化的分布中采样

#### 数学表示
$$P_{\text{top-k}}(w_i) = \begin{cases}
\frac{P(w_i)}{\sum_{j \in \text{top-k}} P(w_j)} & \text{if } w_i \in \text{top-k} \\
0 & \text{otherwise}
\end{cases}$$

In [8]:
def greedy_sampling(logits):
    """贪心采样"""
    return torch.argmax(logits, dim=-1)

def top_k_sampling(logits, k=50, temperature=1.0):
    """Top-k采样"""
    # 应用温度
    logits = logits / temperature
    
    # 获取top-k的值和索引
    top_k_logits, top_k_indices = torch.topk(logits, k, dim=-1)
    
    # 创建掩码，将非top-k的位置设为负无穷
    logits_masked = torch.full_like(logits, float('-inf'))
    logits_masked.scatter_(-1, top_k_indices, top_k_logits)
    
    # 计算概率并采样
    probs = F.softmax(logits_masked, dim=-1)
    return torch.multinomial(probs, 1).squeeze(-1)

## 5. 练习题

### 理论题

1. **解释双向编码器自回归解码器架构的优势，并与纯自回归模型进行比较。**

2. **教师强制和学生强制各有什么优缺点？在什么情况下应该使用哪种策略？**

3. **Top-k采样和Top-p采样的区别是什么？各自适用于什么场景？**

4. **温度参数如何影响文本生成的质量和多样性？**

### 编程题

1. **实现一个支持多种采样策略的文本生成器**
2. **比较不同训练策略对模型性能的影响**
3. **实现一个简单的文本摘要系统**
4. **设计一个评估生成文本质量的指标**

### 思考题

1. **如何平衡生成文本的流畅性和多样性？**
2. **如何控制生成文本的长度和结构？**
3. **如何评估NLG系统的性能？**
4. **未来NLG技术的发展方向是什么？**

### 实践项目

1. **构建一个新闻摘要系统**
2. **开发一个创意写作助手**
3. **实现一个多轮对话机器人**
4. **设计一个代码生成工具**